#1 General Setup

In [ ]:
# Install required libraries
!pip install torch transformers datasets loralib

#Example 6.4 Applying LoRA

In [ ]:
#1: Load the Pre-Trained Model
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Load pre-trained BERT model and tokenizer (for sequence classification)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # num_labels=2 for binary classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#2: Introduce LoRA into the Model
import loralib as lora
import torch.nn as nn

# Function to apply LoRA to the model's linear layers
def apply_lora_to_bert(model, rank=8):
    # Collect layers to replace
    layers_to_replace = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            # Collect the name and module for replacement
            layers_to_replace.append((name, module))

    # Replace the collected linear layers with LoRA-enhanced layers
    for name, module in layers_to_replace:
        lora_layer = lora.Linear(module.in_features, module.out_features, r=rank)
        lora_layer.weight.data = module.weight.data
        if module.bias is not None:
            lora_layer.bias.data = module.bias.data

        # Navigate the model to the correct submodule
        parent_module = model
        sub_names = name.split(".")
        for sub_name in sub_names[:-1]:  # Iterate to the last submodule
            parent_module = getattr(parent_module, sub_name)

        # Replace the final submodule
        setattr(parent_module, sub_names[-1], lora_layer)

# Apply LoRA to the BERT model
apply_lora_to_bert(model, rank=8)


In [ ]:
#3 Prepare the Dataset
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset('imdb')
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']

# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)

# Prepare dataset for PyTorch
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_labels)
)

In [ ]:
#4 Fine-Tune the Model with LoRA
from torch.utils.data import DataLoader
from transformers import AdamW

# Prepare the data loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Set up the optimizer to update the LoRA-enhanced model parameters
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop
model.train()
for epoch in range(1):  # Train for 1 epochs
    for batch in train_loader:
        optimizer.zero_grad()

        # Extract input_ids and labels from batch
        input_ids, labels = batch

        # Pass input_ids and labels to the model
        outputs = model(input_ids=input_ids, labels=labels)  # outputs will contain loss

        # Extract loss
        loss = outputs.loss

        if loss is not None:  # Ensure loss is not None
            loss.backward()  # Backpropagate
            optimizer.step()
            print(f'Epoch {epoch}, Loss: {loss.item()}')
        else:
            print("Loss is None")


Epoch 0, Loss: 0.73193359375
Epoch 0, Loss: 0.69873046875
Epoch 0, Loss: 0.66162109375


In [ ]:
#5: Evaluate the Fine-Tuned Model
from sklearn.metrics import accuracy_score

# Prepare the validation data
val_texts = dataset['test']['text']
val_labels = dataset['test']['label']

# Tokenize validation text
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Prepare dataset for PyTorch
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_labels)
)

# Load the validation data
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Evaluate the model
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids, labels = batch
        outputs = model(input_ids)
        predictions.extend(torch.argmax(outputs.logits, axis=1).tolist())
        true_labels.extend(labels.tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

#Example 6.6 Applying QLoRA (Quantized LoRA)

In [ ]:
#1 Install the libraries
!pip install torch transformers bitsandbytes loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
#2: Load the Pre-Trained Model
import torch
from transformers import BertModel, BertTokenizer, BitsAndBytesConfig

# Load the pre-trained BERT model with 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = BertModel.from_pretrained('bert-base-uncased', quantization_config=bnb_config)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
#3: Apply LoRA to the Model
import loralib as lora

# Apply LoRA to the model (modify the rank as needed)
lora.mark_only_lora_as_trainable(model)
lora.apply_lora(model, rank=8)  # rank=8 for low-rank adaptation


In [ ]:
#4: Prepare Data for Fine-Tuning
from datasets import load_dataset
from transformers import AdamW
from torch.utils.data import DataLoader

# Load the IMDb dataset
dataset = load_dataset('imdb')
train_texts = dataset['train']['text'][:1000]
train_labels = dataset['train']['label'][:1000]

# Tokenize the dataset
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)

# Prepare a DataLoader
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_labels)
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
#5: Fine-Tune the Model
# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop
model.train()
for epoch in range(1):  # Train for 1 epoch (can be adjusted)
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, labels = batch
        outputs = model(input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
#6: Evaluate the Model
from sklearn.metrics import accuracy_score

# Load the validation data
val_texts = dataset['test']['text'][:500]
val_labels = dataset['test']['label'][:500]
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_labels)
)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Evaluate the model
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids, labels = batch
        outputs = model(input_ids)
        predictions.extend(torch.argmax(outputs.logits, axis=1).tolist())
        true_labels.extend(labels.tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')
